In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
import pandas as pd
import seaborn as sns

In [4]:
training_set=image_dataset_from_directory(
    "./train",
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(224,224),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)

Found 76515 files belonging to 4 classes.


In [5]:
validating_set=image_dataset_from_directory(
    "./val",
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(224,224),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)

Found 21861 files belonging to 4 classes.


In [31]:
##Entrainer le model
from tensorflow import keras
INPUT_SHAPE=(224,224,3)
mobileNetModel=keras.applications.MobileNetV3Large(
    input_shape = INPUT_SHAPE,
    alpha=1.0,
    minimalistic=False,
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    classes=1000,
    pooling=None,
    dropout_rate=0.2,
    classifier_activation="softmax",
    include_preprocessing=True,
    name="MobileNetV3"
)

In [32]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.metrics import F1Score

model=Sequential()

model.add(Input(shape=INPUT_SHAPE))
model.add(mobileNetModel)
model.add(Dense(units=4,activation="softmax"))

metrics=["accuracy" ,F1Score]

In [33]:
from tensorflow.keras.optimizers import AdamW
model.compile(optimizer=AdamW(learning_rate=0.01),loss="categorical_crossentropy",metrics=metrics)


In [34]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ MobileNetV3 (Functional)        │ (None, 1000)           │     5,507,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │         4,004 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,511,436 (21.02 MB)

 Trainable params: 5,487,036 (20.93 MB)

 Non-trainable params: 24,400 (95.31 KB)

In [35]:
model.load_weights("Trained_Model.h5")

In [38]:
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
import numpy as np
import tensorflow as tf

img_path = './test/NORMAL/NORMAL-9251-5.jpeg'
img = tf.keras.utils.load_img(img_path, target_size=(224, 224))
x = tf.keras.utils.img_to_array(img)
x = np.expand_dims(x, axis=0)

# MobileNetV3 preprocessing
x = preprocess_input(x)

# Make prediction
preds = model.predict(x)
classes = ['CNV', 'DME', 'DRUSEN', 'NORMAL']
predicted_class = classes[np.argmax(preds)]
print("Predicted class:", predicted_class)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
Predicted class: NORMAL
